# Ejercicios de Limpieza y Preparación de Datos

Para la realización de la siguiente guía práctica utilizaremos el dataset [Netflix Movies and TV Shows](https://www.kaggle.com/datasets/shivamb/netflix-shows).

> *El archivo ya descargado se encuentra bajo la dirección `./datasets/Alumnos/Equipo Faith No More/Netflix Movies and TV Shows/netflix_titles.csv`*

In [1]:
# ~~~ Importación de librerías necesarias y del dataset ~~~ #

import pandas as pd
import numpy as np

direccion = r"../../../datasets/Alumnos/Equipo Faith No More/Netflix Movies and TV Shows/netflix_titles.csv"

df = pd.read_csv(direccion)

# Verificamos que el archivo se haya cargado correctamente y vemos la estructura del dataset
print(df.head())
print(df.info())

  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021        

## Ejercicio n°1: Relleno de valores faltantes

Descripción: Algunas películas/series no tienen información de director. Podemos rellenar esos valores faltantes con la cadena "Desconocido".

Pregunta: ¿Cuántos valores faltantes de la columna director quedan después de imputar?

Resolución:

In [2]:
filas = df['director'].isna().sum()
print(f"La columna 'director' tiene {filas} filas vacias")

df['director'] = df['director'].fillna("Desconocido")
filas = df['director'].isna().sum()
print(f"La columna 'director' tiene {filas} filas vacias")

La columna 'director' tiene 2634 filas vacias
La columna 'director' tiene 0 filas vacias


## Ejercicio n°2: Normalización de texto

Descripción: La columna country presenta valores con variaciones de mayúsculas/minúsculas. Vamos a estandarizarlos para que todo quede en mayúsculas.

Pregunta: ¿Cuál es el conteo de películas por país después de la normalización?

Resolución:

In [3]:
df['country'] = df['country'].str.upper()
df['country'].value_counts().head(10)

country
UNITED STATES     2818
INDIA              972
UNITED KINGDOM     419
JAPAN              245
SOUTH KOREA        199
CANADA             181
SPAIN              145
FRANCE             124
MEXICO             110
EGYPT              106
Name: count, dtype: int64

## Ejercicio n°3: Conversión de fechas

Descripción: La columna date_added es de tipo object. Debemos convertirla a tipo datetime para trabajar con series temporales.

Pregunta: ¿Cuál es la fecha más reciente y la más antigua de adición a Netflix?

Resolución:

In [4]:
df['date_added'] = pd.to_datetime(df['date_added'].str.strip())
df['date_added'].min(), df['date_added'].max()

(Timestamp('2008-01-01 00:00:00'), Timestamp('2021-09-25 00:00:00'))

## Ejercicio n°4: Creación de variables derivadas

Descripción: A partir de la columna listed_in, que contiene géneros separados por coma, crearemos una variable binaria indicando si el título es una “Documentary”.

Pregunta: ¿Cuántos documentales hay en el catálogo de Netflix?

Resolución:

In [5]:
df['is_documentary'] = df['listed_in'].str.contains("Documentaries", case=False, na=False)
documentales = df['is_documentary'].sum()
print(f"En Netflix hay {documentales} documentales.")

En Netflix hay 869 documentales.


## Ejercicio n°5: Creación de variables dummies

Descripción: La columna type contiene dos categorías: Movie y TV Show. Vamos a convertir esta variable categórica en variables dummy.

Pregunta: ¿Cómo se ve el dataset después de aplicar get_dummies a la columna type?

Resolución:

In [6]:
df_dummies = pd.get_dummies(df, columns=['type'], drop_first=False)
df_dummies[['type_Movie', 'type_TV Show']].head()

,type_Movie,type_TV Show
0,True,False
1,False,True
2,False,True
3,False,True
4,False,True


## Ejercicio n°6: Categorización explícita

Descripción: La columna rating contiene categorías como PG, R, TV-MA, etc. Vamos a convertirla en un tipo Categorical para optimizar el almacenamiento y facilitar agrupamientos.

Pregunta: ¿Qué categorías únicas existen en rating y cuál es la más frecuente?

Resolución:

In [7]:
df['rating'] = df['rating'].astype('category')
conteo_rating = df['rating'].value_counts()
print(conteo_rating)

rating
TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
66 min         1
84 min         1
74 min         1
Name: count, dtype: int64


## Ejercicio n°7: Correción de ratings

A partir del resultado del ejercicio anterior vemos que hay 3 películas que tienen errores en el rating: 

In [8]:
# Definir la lista oficial de ratings
df['rating'] = df['rating'].str.strip()
ratings_validos = ['TV-MA','TV-14','TV-PG','R','PG-13','TV-Y7','TV-Y','PG','TV-G','NR','G','TV-Y7-FV','NC-17','UR']

# Filtrar filas cuyo rating no esté en la lista oficial
malos_ratings = df[~df['rating'].isin(ratings_validos)]

# Mostrar las filas
malos_ratings

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,is_documentary
5541,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,UNITED STATES,2017-04-04,2017,74 min,NaN,Movies,"Louis C.K. muses on religion, eternal love, gi...",False
5794,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,UNITED STATES,2016-09-16,2010,84 min,NaN,Movies,Emmy-winning comedy writer Louis C.K. brings h...,False
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,UNITED STATES,2016-08-15,2015,66 min,NaN,Movies,The comic puts his trademark hilarious/thought...,False
5989,s5990,Movie,13TH: A Conversation with Oprah Winfrey & Ava ...,Desconocido,"Oprah Winfrey, Ava DuVernay",NaN,2017-01-26,2017,NaN,37 min,Movies,Oprah Winfrey sits down with director Ava DuVe...,False
6827,s6828,TV Show,Gargantia on the Verdurous Planet,Desconocido,"Kaito Ishikawa, Hisako Kanemoto, Ai Kayano, Ka...",JAPAN,2016-12-01,2013,NaN,1 Season,"Anime Series, International TV Shows","After falling through a wormhole, a space-dwel...",False
7312,s7313,TV Show,Little Lunch,Desconocido,"Flynn Curry, Olivia Deeble, Madison Lu, Oisín ...",AUSTRALIA,2018-02-01,2015,NaN,1 Season,"Kids' TV, TV Comedies","Adopting a child's perspective, this show take...",False
7537,s7538,Movie,My Honor Was Loyalty,Alessandro Pepe,"Leone Frisa, Paolo Vaccarino, Francesco Miglio...",ITALY,2017-03-01,2015,NaN,115 min,Dramas,"Amid the chaos and horror of World War II, a c...",False


La estrategia es poner en NaN aquellas películas que tengan un valor erróneo en el rating.

In [9]:
# Limpiar valores inválidos: los que no estén en ratings_validos se convierten en NaN
df['rating'] = df['rating'].where(df['rating'].isin(ratings_validos), np.nan)

# Verificar que ya no hay valores incorrectos
df['rating'].value_counts(dropna=False)

rating
TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
NaN            7
TV-Y7-FV       6
NC-17          3
UR             3
Name: count, dtype: int64